<a href="https://colab.research.google.com/github/HoangShiya/HUET/blob/main/TH%E1%BB%B0C_H%C3%80NH_PH%C3%82N_T%C3%8DCH_ANOVA_1_CHI%E1%BB%80U.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# THỰC HÀNH PHÂN TÍCH ANOVA 1 CHIỀU
---
---

Sử dụng các thư viện cần thiết

In [2]:
from google.colab import drive
import pandas as pd
import statistics as sts
import numpy as np 
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns

## **PHÂN TÍCH ANOVA 1 CHIỀU VỚI TẬP DỮ LIỆU "Excavation Depth and Archaeology"**

> **Bước 1**: Tiến hành lấy tập dữ liệu owan01.csv từ drive để tiến hành phân tích

In [3]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Python/owan01.csv')
print(f"Dữ liệu của tập dữ liệu owan01.csv:\n ----------------------------------\n {df}")

Mounted at /content/drive
Dữ liệu của tập dữ liệu owan01.csv:
 ----------------------------------
      X1    X2     X3    X4
0    93  85.0  100.0  96.0
1   120  45.0   75.0  58.0
2    65  80.0   65.0  95.0
3   105  28.0   40.0  90.0
4   115  75.0   73.0  65.0
5    82  70.0   65.0  80.0
6    99  65.0   50.0  85.0
7    87  55.0   30.0  95.0
8   100  50.0   45.0  82.0
9    90  40.0   50.0   NaN
10   78   NaN   45.0   NaN
11   95   NaN   55.0   NaN
12   93   NaN    NaN   NaN
13   88   NaN    NaN   NaN
14  110   NaN    NaN   NaN


Chuyển đổi df thành dạng phù hợp để phân tích ANOVA một chiều theo gói scipy.stats, trường hợp này có thể chuyển dữ liệu sang dạng list và  sử dụng hàm *dropna()*

> **Cú pháp**: dataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

*Chú thích*:

1) *axis*: 0 là "index", 1 là "column", mặc định axis = 0

Xác định xem các hàng hoặc cột chứa các giá trị bị thiếu có bị xóa hay không.

*   0, hoặc "index": Thả các hàng chứa các giá trị bị thiếu.
*   1 hoặc "column": Bỏ các cột chứa giá trị bị thiếu.

2) *how*: "any" hoặc "all", mặc định là any

Xác định xem hàng hoặc cột có bị xóa khỏi DataFrame hay không khi chúng ta có ít nhất một NA hoặc tất cả NA.

*   "any": Nếu có bất kỳ giá trị NA nào sẽ loại bỏ luôn hàng hoặc cột đó.
*   "all": Nếu tất cả các giá trị đều là NA sẽ loại luôn bỏ hàng hoặc cột đó.

3) *thresh*: int, optional

Yêu cầu nhiều giá trị không phải NA

4) *subset*: Nhãn cột (column label), chuỗi nhãn (sequence label) hoặc tùy chọn (optional)

Xem xét các nhãn dọc theo trục khác nhau.

Ví dụ: Nếu bạn đang bỏ hàng thì đây sẽ là danh sách bao gồm các cột cần thiết.

5) *inplace*: bool, mặc định là "sai" (False)

Nếu Đúng, hãy thực hiện thao tác tại chỗ và trả về giá trị None.

> **Chức năng**: Loại bỏ những giá trị bị thiếu

> **Giá trị trả về**: Dataframe hoặc None

DataFrame với các mục NA bị loại bỏ khỏi nó hoặc Không có nếu inplace=True.


In [4]:
# Khởi tạo giá trị alpha = 0.05 là ý nghĩa của quá trình kiểm định
alpha = 0.05

# Khởi tạo biến sample_converted_by_df dùng để lưu trữ dữ liệu của biến df khi được chuyển đổi sang dạng list và sàng lọc dữ liệu
sample_converted_by_df = [list(df["X1"].dropna()), 
                          list(df["X2"].dropna()),
                          list(df["X3"].dropna()),
                          list(df["X4"].dropna())]

print(f"Dữ liệu của tập dữ liệu owan01.csv khi được chuyển đổi và xử lí số liệu:\n --------------------------------------------------------------------\n {sample_converted_by_df}")

Dữ liệu của tập dữ liệu owan01.csv khi được chuyển đổi và xử lí số liệu:
 --------------------------------------------------------------------
 [[93, 120, 65, 105, 115, 82, 99, 87, 100, 90, 78, 95, 93, 88, 110], [85.0, 45.0, 80.0, 28.0, 75.0, 70.0, 65.0, 55.0, 50.0, 40.0], [100.0, 75.0, 65.0, 40.0, 73.0, 65.0, 50.0, 30.0, 45.0, 50.0, 45.0, 55.0], [96.0, 58.0, 95.0, 90.0, 65.0, 80.0, 85.0, 95.0, 82.0]]


### a) Kiểm định các nhóm có tuân theo phân phối chuẩn không

> **Bước 2.a**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Nhóm tuân theo phân phối chuẩn}$$

Giả thuyết $H_a$:

$$H_a: \text{Nhóm không tuân theo phân phối chuẩn}$$

> **Bước 3.a**: Tiến hành kiểm định và tính toán các đại lượng cần thiết

> **Bước 4.a**: Kết luận dựa trên kết quả kiểm định

In [5]:
# Để kiểm định từng hàng, ta sử dụng vòng lặp for và so sánh giá trị p với mức ý nghĩa để tiến hành kiểm định
for i in range(0, 4, 1):
  p = st.shapiro(sample_converted_by_df[i])[1]

  # Phát biểu kết quả kiểm định từng nhóm
  print("-------------------------------------------------------------------------")
  print(f"Giá trị p của nhóm thứ {i+1} là: {np.round(p, 2)}")
  if (p < alpha):
    print(f"Vì giá trị p của nhóm thứ {i+1} bé hơn mức ý nghĩa của bài toán ({np.round(p, 2)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
  else:
    print(f"Vì giá trị p của nhóm thứ {i+1} không bé hơn mức ý nghĩa của bài toán ({np.round(p, 2)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

print("-------------------------------------------------------------------------")

-------------------------------------------------------------------------
Giá trị p của nhóm thứ 1 là: 1.0
Vì giá trị p của nhóm thứ 1 không bé hơn mức ý nghĩa của bài toán (1.0 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p của nhóm thứ 2 là: 0.86
Vì giá trị p của nhóm thứ 2 không bé hơn mức ý nghĩa của bài toán (0.86 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p của nhóm thứ 3 là: 0.6
Vì giá trị p của nhóm thứ 3 không bé hơn mức ý nghĩa của bài toán (0.6 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p của nhóm thứ 4 là: 0.14
Vì giá trị p của nhóm thứ 4 không bé hơn mức ý nghĩa của bài toán (0.14 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------


### b) Kiểm định sự đồng nhất giữa phương sai các nhóm

> **Bước 2.b**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Không có sự khác biệt về phương sai của các nhóm}$$

Giả thuyết $H_a$:

$$H_a: \text{Có sự khác biệt về phương sai của các nhóm}$$

> **Bước 3.b**: Tiến hành kiểm định và tính toán các đại lượng cần thiết


In [6]:
# Khởi tạo biến stt, p để lưu giá trị của stat value và p value
stt, p = st.levene(sample_converted_by_df[0], sample_converted_by_df[1], sample_converted_by_df[2], sample_converted_by_df[3])   

# Xuất kết quả stat value, p value ra màn hình
print(f"Giá trị stat: {np.round(stt, 2)}")
print(f"Giá trị p: {np.round(p, 2)}")

Giá trị stat: 0.8
Giá trị p: 0.5


> **Bước 4.b**: Kết luận dựa trên kết quả kiểm định

In [7]:
if (p < alpha):
  print(f"Vì giá trị p  bé hơn mức ý nghĩa của bài toán ({np.round(p, 2)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
else:
  print(f"Vì giá trị p không bé hơn mức ý nghĩa của bài toán ({np.round(p, 2)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

Vì giá trị p không bé hơn mức ý nghĩa của bài toán (0.5 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0


Vì các nhóm độc lập, tuân theo phân phối chuẩn và có sự đồng nhất về phương sai giữa các nhóm nên ta **có thể tiến hành kiểm định ANOVA**

### TIẾN HÀNH PHÂN TÍCH ANOVA 1 CHIỀU

> **Bước 1**: Lấy dữ liệu và tính toán các thông số cần thiết

*   Dữ liệu được sử dụng là sample_converted_by_df (Dữ liệu đã được xử lí từ owan01.csv)
*   alpha = 0.05 là ý nghĩa của bài toán trong quá trình kiểm định

> **Bước 2**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Không có sự khác biệt về trung bình độ sâu của các nhóm}$$

Giả thuyết $H_a$:

$$H_a: \text{Có sự khác biệt về trung bình độ sâu của các nhóm}$$

> **Bước 3**: Tiến hành kiểm định ANOVA 1 chiều

In [16]:
# Khởi tạo 2 biến f và p dùng để lưu trữ giá trị f stat và p value
f, p = st.f_oneway(sample_converted_by_df[0], sample_converted_by_df[1], sample_converted_by_df[2], sample_converted_by_df[3])

# Xuất giá trị p ra màn hình
print(f"Giá trị p khi tiến hành kiểm định: {np.round(p, 10)}")

Giá trị p khi tiến hành kiểm định: 7.991e-07


> **Bước 4**: Kết luận dựa trên kết quả kiểm định

In [17]:
if (p < alpha):
  print(f"Vì giá trị p  bé hơn mức ý nghĩa của bài toán ({np.round(p, 10)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
else:
  print(f"Vì giá trị p không bé hơn mức ý nghĩa của bài toán ({np.round(p, 10)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

Vì giá trị p  bé hơn mức ý nghĩa của bài toán (7.991e-07 < 0.05). Vì vậy không có bằng chứng bác bỏ giả thuyết H0


## **PHÂN TÍCH ANOVA 1 CHIỀU VỚI TẬP DỮ LIỆU "Apple orchard experiment"**

> **Bước 1**: Tiến hành lấy tập dữ liệu owan02.csv từ drive để tiến hành phân tích

In [24]:
df2 = pd.read_csv('/content/drive/MyDrive/Python/owan02.csv')
print(f"Dữ liệu của tập dữ liệu owan02.csv:\n ----------------------------------\n {df2}")

Dữ liệu của tập dữ liệu owan02.csv:
 ----------------------------------
      X1    X2    X3    X4    X5
0  2569  2074  2505  2838  1532
1  2928  2885  2315  2351  2552
2  2865  3378  2667  3001  3083
3  3844  3906  2390  2439  2330
4  3027  2782  3021  2199  2079
5  2336  3018  3085  3318  3366
6  3211  3383  3308  3601  2416
7  3037  3447  3231  3291  3100


In [38]:
# Khởi tạo giá trị alpha = 0.05 là ý nghĩa của quá trình kiểm định
alpha = 0.05

# Khởi tạo biến sample_converted_by_df dùng để lưu trữ dữ liệu của biến df khi được chuyển đổi sang dạng list và sàng lọc dữ liệu
sample_converted_by_df2 = [list(df2["X1"].dropna()), 
                           list(df2["X2"].dropna()),
                           list(df2["X3"].dropna()),
                           list(df2["X4"].dropna()),
                           list(df2["X5"].dropna())]

print(f"Dữ liệu của tập dữ liệu owan02.csv khi được chuyển đổi và xử lí số liệu:\n --------------------------------------------------------------------\n {sample_converted_by_df2}")

Dữ liệu của tập dữ liệu owan02.csv khi được chuyển đổi và xử lí số liệu:
 --------------------------------------------------------------------
 [[2569, 2928, 2865, 3844, 3027, 2336, 3211, 3037], [2074, 2885, 3378, 3906, 2782, 3018, 3383, 3447], [2505, 2315, 2667, 2390, 3021, 3085, 3308, 3231], [2838, 2351, 3001, 2439, 2199, 3318, 3601, 3291], [1532, 2552, 3083, 2330, 2079, 3366, 2416, 3100]]


### a) Kiểm định các nhóm có tuân theo phân phối chuẩn không

> **Bước 2.a**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Nhóm tuân theo phân phối chuẩn}$$

Giả thuyết $H_a$:

$$H_a: \text{Nhóm không tuân theo phân phối chuẩn}$$

> **Bước 3.a**: Tiến hành kiểm định và tính toán các đại lượng cần thiết

> **Bước 4.a**: Kết luận dựa trên kết quả kiểm định

In [30]:
# Để kiểm định từng hàng, ta sử dụng vòng lặp for và so sánh giá trị p với mức ý nghĩa để tiến hành kiểm định
for i in range(0, 5, 1):
  p2 = st.shapiro(sample_converted_by_df2[i])[1]

  # Phát biểu kết quả kiểm định từng nhóm
  print("-------------------------------------------------------------------------")
  print(f"Giá trị p2 của nhóm thứ {i+1} là: {np.round(p2, 2)}")
  if (p2 < alpha):
    print(f"Vì giá trị p2 của nhóm thứ {i+1} bé hơn mức ý nghĩa của bài toán ({np.round(p2, 2)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
  else:
    print(f"Vì giá trị p2 của nhóm thứ {i+1} không bé hơn mức ý nghĩa của bài toán ({np.round(p2, 2)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

print("-------------------------------------------------------------------------")

-------------------------------------------------------------------------
Giá trị p của nhóm thứ 1 là: 0.64
Vì giá trị p của nhóm thứ 1 không bé hơn mức ý nghĩa của bài toán (0.64 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p của nhóm thứ 2 là: 0.75
Vì giá trị p của nhóm thứ 2 không bé hơn mức ý nghĩa của bài toán (0.75 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p của nhóm thứ 3 là: 0.32
Vì giá trị p của nhóm thứ 3 không bé hơn mức ý nghĩa của bài toán (0.32 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p của nhóm thứ 4 là: 0.6
Vì giá trị p của nhóm thứ 4 không bé hơn mức ý nghĩa của bài toán (0.6 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------


### b) Kiểm định sự đồng nhất giữa phương sai các nhóm

> **Bước 2.b**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Không có sự khác biệt về phương sai của các nhóm}$$

Giả thuyết $H_a$:

$$H_a: \text{Có sự khác biệt về phương sai của các nhóm}$$

> **Bước 3.b**: Tiến hành kiểm định và tính toán các đại lượng cần thiết


In [31]:
# Khởi tạo biến stt2, p2 để lưu giá trị của stat2 value và p2 value
stt2, p2 = st.levene(sample_converted_by_df2[0], sample_converted_by_df2[1], sample_converted_by_df2[2], sample_converted_by_df2[3], sample_converted_by_df2[4])   

# Xuất kết quả stat3 value, p2 value ra màn hình
print(f"Giá trị stat2: {np.round(stt2, 2)}")
print(f"Giá trị p2: {np.round(p2, 2)}")

Giá trị stat2: 0.43
Giá trị p2: 0.79


> **Bước 4.b**: Kết luận dựa trên kết quả kiểm định

In [32]:
if (p2 < alpha):
  print(f"Vì giá trị p2  bé hơn mức ý nghĩa của bài toán ({np.round(p2, 2)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
else:
  print(f"Vì giá trị p2 không bé hơn mức ý nghĩa của bài toán ({np.round(p2, 2)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

Vì giá trị p2 không bé hơn mức ý nghĩa của bài toán (0.79 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0


Vì các nhóm độc lập, tuân theo phân phối chuẩn và có sự đồng nhất về phương sai giữa các nhóm nên ta **có thể tiến hành kiểm định ANOVA**

### TIẾN HÀNH PHÂN TÍCH ANOVA 1 CHIỀU

> **Bước 1**: Lấy dữ liệu và tính toán các thông số cần thiết

*   Dữ liệu được sử dụng là sample_converted_by_df2 (Dữ liệu đã được xử lí từ owan02.csv)
*   alpha = 0.05 là ý nghĩa của bài toán trong quá trình kiểm định

> **Bước 2**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Không có sự khác biệt về trung bình độ tăng trưởng của các nhóm}$$

Giả thuyết $H_a$:

$$H_a: \text{có sự khác biệt về trung bình độ tăng trưởng của các nhóm}$$

> **Bước 3**: Tiến hành kiểm định ANOVA 1 chiều

In [35]:
# Khởi tạo 2 biến f2 và p2 dùng để lưu trữ giá trị f2 stat và p2 value
f2, p2 = st.f_oneway(sample_converted_by_df2[0], sample_converted_by_df2[1], sample_converted_by_df2[2], sample_converted_by_df2[3], sample_converted_by_df2[4])

# Xuất giá trị p2 ra màn hình
print(f"Giá trị p2 khi tiến hành kiểm định: {np.round(p2, 2)}")

Giá trị p khi tiến hành kiểm định: 0.8


> **Bước 4**: Kết luận dựa trên kết quả kiểm định

In [36]:
if (p2 < alpha):
  print(f"Vì giá trị p2  bé hơn mức ý nghĩa của bài toán ({np.round(p2, 2)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
else:
  print(f"Vì giá trị p2 không bé hơn mức ý nghĩa của bài toán ({np.round(p2, 2)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

Vì giá trị p2 không bé hơn mức ý nghĩa của bài toán (0.28 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0


## **PHÂN TÍCH ANOVA 1 CHIỀU VỚI TẬP DỮ LIỆU "Red dye number 40"**

> **Bước 1**: Tiến hành lấy tập dữ liệu owan03.csv từ drive để tiến hành phân tích

In [37]:
df3 = pd.read_csv('/content/drive/MyDrive/Python/owan03.csv')
print(f"Dữ liệu của tập dữ liệu owan03.csv:\n ----------------------------------\n {df3}")

Dữ liệu của tập dữ liệu owan03.csv:
 ----------------------------------
      X1    X2    X3     X4
0    70  49.0  30.0   34.0
1    77  60.0  37.0   36.0
2    83  63.0  56.0   48.0
3    87  67.0  65.0   48.0
4    92  70.0  76.0   65.0
5    93  74.0  83.0   91.0
6   100  77.0  87.0   98.0
7   102  80.0  90.0  102.0
8   102  89.0  94.0    NaN
9   103   NaN  97.0    NaN
10   96   NaN   NaN    NaN


In [53]:
# Khởi tạo giá trị alpha = 0.05 là ý nghĩa của quá trình kiểm định
alpha = 0.05

# Khởi tạo biến sample_converted_by_df dùng để lưu trữ dữ liệu của biến df khi được chuyển đổi sang dạng list và sàng lọc dữ liệu
sample_converted_by_df3 = [list(df3["X1"].dropna()), 
                           list(df3["X2"].dropna()),
                           list(df3["X3"].dropna()),
                           list(df3["X4"].dropna())]

print(f"Dữ liệu của tập dữ liệu owan03.csv khi được chuyển đổi và xử lí số liệu:\n --------------------------------------------------------------------\n {sample_converted_by_df3}")

Dữ liệu của tập dữ liệu owan03.csv khi được chuyển đổi và xử lí số liệu:
 --------------------------------------------------------------------
 [[70, 77, 83, 87, 92, 93, 100, 102, 102, 103, 96], [49.0, 60.0, 63.0, 67.0, 70.0, 74.0, 77.0, 80.0, 89.0], [30.0, 37.0, 56.0, 65.0, 76.0, 83.0, 87.0, 90.0, 94.0, 97.0], [34.0, 36.0, 48.0, 48.0, 65.0, 91.0, 98.0, 102.0]]


### a) Kiểm định các nhóm có tuân theo phân phối chuẩn không

> **Bước 2.a**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Nhóm tuân theo phân phối chuẩn}$$

Giả thuyết $H_a$:

$$H_a: \text{Nhóm không tuân theo phân phối chuẩn}$$

> **Bước 3.a**: Tiến hành kiểm định và tính toán các đại lượng cần thiết

> **Bước 4.a**: Kết luận dựa trên kết quả kiểm định

In [41]:
# Để kiểm định từng hàng, ta sử dụng vòng lặp for và so sánh giá trị p với mức ý nghĩa để tiến hành kiểm định
for i in range(0, 4, 1):
  p3 = st.shapiro(sample_converted_by_df3[i])[1]

  # Phát biểu kết quả kiểm định từng nhóm
  print("-------------------------------------------------------------------------")
  print(f"Giá trị p3 của nhóm thứ {i+1} là: {np.round(p3, 2)}")
  if (p3 < alpha):
    print(f"Vì giá trị p3 của nhóm thứ {i+1} bé hơn mức ý nghĩa của bài toán ({np.round(p3, 2)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
  else:
    print(f"Vì giá trị p3 của nhóm thứ {i+1} không bé hơn mức ý nghĩa của bài toán ({np.round(p3, 2)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

print("-------------------------------------------------------------------------")

-------------------------------------------------------------------------
Giá trị p3 của nhóm thứ 1 là: 0.24
Vì giá trị p3 của nhóm thứ 1 không bé hơn mức ý nghĩa của bài toán (0.24 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p3 của nhóm thứ 2 là: 0.75
Vì giá trị p3 của nhóm thứ 2 không bé hơn mức ý nghĩa của bài toán (0.75 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p3 của nhóm thứ 3 là: 0.32
Vì giá trị p3 của nhóm thứ 3 không bé hơn mức ý nghĩa của bài toán (0.32 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p3 của nhóm thứ 4 là: 0.6
Vì giá trị p3 của nhóm thứ 4 không bé hơn mức ý nghĩa của bài toán (0.6 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
------------------------------------------------------------------

### b) Kiểm định sự đồng nhất giữa phương sai các nhóm

> **Bước 2.b**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Không có sự khác biệt về phương sai của các nhóm}$$

Giả thuyết $H_a$:

$$H_a: \text{Có sự khác biệt về phương sai của các nhóm}$$

> **Bước 3.b**: Tiến hành kiểm định và tính toán các đại lượng cần thiết


In [43]:
# Khởi tạo biến stt3, p3 để lưu giá trị của stat3 value và p3 value
stt3, p3 = st.levene(sample_converted_by_df3[0], sample_converted_by_df3[1], sample_converted_by_df3[2], sample_converted_by_df3[3])   

# Xuất kết quả stat3 value, p3 value ra màn hình
print(f"Giá trị stat3: {np.round(stt3, 2)}")
print(f"Giá trị p3: {np.round(p3, 5)}")

Giá trị stat3: 9.0
Giá trị p3: 0.00019


> **Bước 4.b**: Kết luận dựa trên kết quả kiểm định

In [44]:
if (p3 < alpha):
  print(f"Vì giá trị p3  bé hơn mức ý nghĩa của bài toán ({np.round(p3, 5)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
else:
  print(f"Vì giá trị p3 không bé hơn mức ý nghĩa của bài toán ({np.round(p3, 5)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

Vì giá trị p3  bé hơn mức ý nghĩa của bài toán (0.00019 < 0.05). Vì vậy không có bằng chứng bác bỏ giả thuyết H0


Vì các nhóm độc lập, tuân theo phân phối chuẩn và có sự đồng nhất về phương sai giữa các nhóm nên ta **có thể tiến hành kiểm định ANOVA**

### TIẾN HÀNH PHÂN TÍCH ANOVA 1 CHIỀU

> **Bước 1**: Lấy dữ liệu và tính toán các thông số cần thiết

*   Dữ liệu được sử dụng là sample_converted_by_df3 (Dữ liệu đã được xử lí từ owan03.csv)
*   alpha = 0.05 là ý nghĩa của bài toán trong quá trình kiểm định

> **Bước 2**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Không có sự khác biệt về trung bình độ tăng trưởng của các nhóm}$$

Giả thuyết $H_a$:

$$H_a: \text{có sự khác biệt về trung bình độ tăng trưởng của các nhóm}$$

> **Bước 3**: Tiến hành kiểm định ANOVA 1 chiều

In [49]:
# Khởi tạo 2 biến f3 và p3 dùng để lưu trữ giá trị f3 stat và p3 value
f3, p3 = st.f_oneway(sample_converted_by_df3[0], sample_converted_by_df3[1], sample_converted_by_df3[2], sample_converted_by_df3[3])

# Xuất giá trị p3 ra màn hình
print(f"Giá trị p3 khi tiến hành kiểm định: {np.round(p3, 20)}")

Giá trị p3 khi tiến hành kiểm định: 1.985e-17


> **Bước 4**: Kết luận dựa trên kết quả kiểm định

In [50]:
if (p3 < alpha):
  print(f"Vì giá trị p3  bé hơn mức ý nghĩa của bài toán ({np.round(p3, 20)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
else:
  print(f"Vì giá trị p3 không bé hơn mức ý nghĩa của bài toán ({np.round(p3, 20)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

Vì giá trị p3  bé hơn mức ý nghĩa của bài toán (1.985e-17 < 0.05). Vì vậy không có bằng chứng bác bỏ giả thuyết H0


## **PHÂN TÍCH ANOVA 1 CHIỀU VỚI TẬP DỮ LIỆU "Business Startup Costs"**

> **Bước 1**: Tiến hành lấy tập dữ liệu owan04.csv từ drive để tiến hành phân tích

In [51]:
df4 = pd.read_csv('/content/drive/MyDrive/Python/owan04.csv')
print(f"Dữ liệu của tập dữ liệu owan04.csv:\n ----------------------------------\n {df4}")

Dữ liệu của tập dữ liệu owan04.csv:
 ----------------------------------
        X1     X2     X3     X4   X5
0    80.0  150.0   48.0  100.0   25
1   125.0   40.0   35.0   96.0   80
2    35.0  120.0   95.0   35.0   30
3    58.0   75.0   45.0   99.0   35
4   110.0  160.0   75.0   75.0   30
5   140.0   60.0  115.0  150.0   28
6    97.0   45.0   42.0   45.0   20
7    50.0  100.0   78.0  100.0   75
8    65.0   86.0   65.0  120.0   48
9    79.0   87.0  125.0   50.0   20
10   35.0   90.0    NaN    NaN   50
11   85.0    NaN    NaN    NaN   75
12  120.0    NaN    NaN    NaN   55
13    NaN    NaN    NaN    NaN   60
14    NaN    NaN    NaN    NaN   85
15    NaN    NaN    NaN    NaN  110


In [52]:
# Khởi tạo giá trị alpha = 0.05 là ý nghĩa của quá trình kiểm định
alpha = 0.05

# Khởi tạo biến sample_converted_by_df4 dùng để lưu trữ dữ liệu của biến df khi được chuyển đổi sang dạng list và sàng lọc dữ liệu
sample_converted_by_df4 = [list(df4["X1"].dropna()), 
                           list(df4["X2"].dropna()),
                           list(df4["X3"].dropna()),
                           list(df4["X4"].dropna()),
                           list(df4["X5"].dropna())]

print(f"Dữ liệu của tập dữ liệu owan04.csv khi được chuyển đổi và xử lí số liệu:\n --------------------------------------------------------------------\n {sample_converted_by_df4}")

Dữ liệu của tập dữ liệu owan04.csv khi được chuyển đổi và xử lí số liệu:
 --------------------------------------------------------------------
 [[80.0, 125.0, 35.0, 58.0, 110.0, 140.0, 97.0, 50.0, 65.0, 79.0, 35.0, 85.0, 120.0], [150.0, 40.0, 120.0, 75.0, 160.0, 60.0, 45.0, 100.0, 86.0, 87.0, 90.0], [48.0, 35.0, 95.0, 45.0, 75.0, 115.0, 42.0, 78.0, 65.0, 125.0], [100.0, 96.0, 35.0, 99.0, 75.0, 150.0, 45.0, 100.0, 120.0, 50.0], [25, 80, 30, 35, 30, 28, 20, 75, 48, 20, 50, 75, 55, 60, 85, 110]]


### a) Kiểm định các nhóm có tuân theo phân phối chuẩn không

> **Bước 2.a**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Nhóm tuân theo phân phối chuẩn}$$

Giả thuyết $H_a$:

$$H_a: \text{Nhóm không tuân theo phân phối chuẩn}$$

> **Bước 3.a**: Tiến hành kiểm định và tính toán các đại lượng cần thiết

> **Bước 4.a**: Kết luận dựa trên kết quả kiểm định

In [54]:
# Để kiểm định từng hàng, ta sử dụng vòng lặp for và so sánh giá trị p với mức ý nghĩa để tiến hành kiểm định
for i in range(0, 5, 1):
  p4 = st.shapiro(sample_converted_by_df4[i])[1]

  # Phát biểu kết quả kiểm định từng nhóm
  print("-------------------------------------------------------------------------")
  print(f"Giá trị p4 của nhóm thứ {i+1} là: {np.round(p4, 2)}")
  if (p4 < alpha):
    print(f"Vì giá trị p4 của nhóm thứ {i+1} bé hơn mức ý nghĩa của bài toán ({np.round(p4, 2)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
  else:
    print(f"Vì giá trị p4 của nhóm thứ {i+1} không bé hơn mức ý nghĩa của bài toán ({np.round(p4, 2)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

print("-------------------------------------------------------------------------")

-------------------------------------------------------------------------
Giá trị p4 của nhóm thứ 1 là: 0.74
Vì giá trị p4 của nhóm thứ 1 không bé hơn mức ý nghĩa của bài toán (0.74 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p4 của nhóm thứ 2 là: 0.56
Vì giá trị p4 của nhóm thứ 2 không bé hơn mức ý nghĩa của bài toán (0.56 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p4 của nhóm thứ 3 là: 0.39
Vì giá trị p4 của nhóm thứ 3 không bé hơn mức ý nghĩa của bài toán (0.39 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p4 của nhóm thứ 4 là: 0.58
Vì giá trị p4 của nhóm thứ 4 không bé hơn mức ý nghĩa của bài toán (0.58 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
----------------------------------------------------------------

### b) Kiểm định sự đồng nhất giữa phương sai các nhóm

> **Bước 2.b**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Không có sự khác biệt về phương sai của các nhóm}$$

Giả thuyết $H_a$:

$$H_a: \text{Có sự khác biệt về phương sai của các nhóm}$$

> **Bước 3.b**: Tiến hành kiểm định và tính toán các đại lượng cần thiết


In [56]:
# Khởi tạo biến stt4, p4 để lưu giá trị của stat4 value và p4 value
stt4, p4 = st.levene(sample_converted_by_df4[0], sample_converted_by_df4[1], sample_converted_by_df4[2], sample_converted_by_df4[3], sample_converted_by_df4[4])   

# Xuất kết quả stat4 value, p4 value ra màn hình
print(f"Giá trị stat4: {np.round(stt4, 2)}")
print(f"Giá trị p4: {np.round(p4, 2)}")

Giá trị stat4: 0.21
Giá trị p4: 0.93


> **Bước 4.b**: Kết luận dựa trên kết quả kiểm định

In [58]:
if (p4 < alpha):
  print(f"Vì giá trị p4  bé hơn mức ý nghĩa của bài toán ({np.round(p4, 2)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
else:
  print(f"Vì giá trị p4 không bé hơn mức ý nghĩa của bài toán ({np.round(p4, 2)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

Vì giá trị p4 không bé hơn mức ý nghĩa của bài toán (0.93 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0


Vì các nhóm độc lập, tuân theo phân phối chuẩn và có sự đồng nhất về phương sai giữa các nhóm nên ta **có thể tiến hành kiểm định ANOVA**

### TIẾN HÀNH PHÂN TÍCH ANOVA 1 CHIỀU

> **Bước 1**: Lấy dữ liệu và tính toán các thông số cần thiết

*   Dữ liệu được sử dụng là sample_converted_by_df4 (Dữ liệu đã được xử lí từ owan04.csv)
*   alpha = 0.05 là ý nghĩa của bài toán trong quá trình kiểm định

> **Bước 2**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Không có sự khác biệt về trung bình độ tăng trưởng của các nhóm}$$

Giả thuyết $H_a$:

$$H_a: \text{có sự khác biệt về trung bình độ tăng trưởng của các nhóm}$$

> **Bước 3**: Tiến hành kiểm định ANOVA 1 chiều

In [60]:
# Khởi tạo 2 biến f4 và p4 dùng để lưu trữ giá trị f4 stat và p4 value
f4, p4 = st.f_oneway(sample_converted_by_df4[0], sample_converted_by_df4[1], sample_converted_by_df4[2], sample_converted_by_df4[3], sample_converted_by_df4[4])

# Xuất giá trị p4 ra màn hình
print(f"Giá trị p4 khi tiến hành kiểm định: {np.round(p4, 2)}")

Giá trị p4 khi tiến hành kiểm định: 0.02


> **Bước 4**: Kết luận dựa trên kết quả kiểm định

In [61]:
if (p4 < alpha):
  print(f"Vì giá trị p4  bé hơn mức ý nghĩa của bài toán ({np.round(p4, 2)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
else:
  print(f"Vì giá trị p4 không bé hơn mức ý nghĩa của bài toán ({np.round(p4, 2)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

Vì giá trị p4  bé hơn mức ý nghĩa của bài toán (0.02 < 0.05). Vì vậy không có bằng chứng bác bỏ giả thuyết H0


## **PHÂN TÍCH ANOVA 1 CHIỀU VỚI TẬP DỮ LIỆU "Weights of Football Players"**

> **Bước 1**: Tiến hành lấy tập dữ liệu owan04.csv từ drive để tiến hành phân tích

In [62]:
df5 = pd.read_csv('/content/drive/MyDrive/Python/owan05.csv')
print(f"Dữ liệu của tập dữ liệu owan05.csv:\n ----------------------------------\n {df5}")

Dữ liệu của tập dữ liệu owan05.csv:
 ----------------------------------
      X1   X2   X3   X4   X5
0   250  260  270  260  247
1   255  271  250  255  249
2   255  258  281  265  255
3   264  263  273  257  247
4   250  267  257  268  244
5   265  254  264  263  245
6   245  255  233  247  249
7   252  250  254  253  260
8   266  248  268  251  217
9   246  240  252  252  208
10  251  254  256  266  228
11  263  275  265  264  253
12  248  270  252  210  249
13  228  225  256  236  223
14  221  222  235  225  221
15  223  230  216  230  228
16  220  225  241  232  271


In [64]:
# Khởi tạo giá trị alpha = 0.05 là ý nghĩa của quá trình kiểm định
alpha = 0.05

# Khởi tạo biến sample_converted_by_df5 dùng để lưu trữ dữ liệu của biến df khi được chuyển đổi sang dạng list và sàng lọc dữ liệu
sample_converted_by_df5 = [list(df5["X1"].dropna()), 
                           list(df5["X2"].dropna()),
                           list(df5["X3"].dropna()),
                           list(df5["X4"].dropna()),
                           list(df5["X5"].dropna())]

print(f"Dữ liệu của tập dữ liệu owan05.csv khi được chuyển đổi và xử lí số liệu:\n --------------------------------------------------------------------\n {sample_converted_by_df5}")

Dữ liệu của tập dữ liệu owan05.csv khi được chuyển đổi và xử lí số liệu:
 --------------------------------------------------------------------
 [[250, 255, 255, 264, 250, 265, 245, 252, 266, 246, 251, 263, 248, 228, 221, 223, 220], [260, 271, 258, 263, 267, 254, 255, 250, 248, 240, 254, 275, 270, 225, 222, 230, 225], [270, 250, 281, 273, 257, 264, 233, 254, 268, 252, 256, 265, 252, 256, 235, 216, 241], [260, 255, 265, 257, 268, 263, 247, 253, 251, 252, 266, 264, 210, 236, 225, 230, 232], [247, 249, 255, 247, 244, 245, 249, 260, 217, 208, 228, 253, 249, 223, 221, 228, 271]]


### a) Kiểm định các nhóm có tuân theo phân phối chuẩn không

> **Bước 2.a**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Nhóm tuân theo phân phối chuẩn}$$

Giả thuyết $H_a$:

$$H_a: \text{Nhóm không tuân theo phân phối chuẩn}$$

> **Bước 3.a**: Tiến hành kiểm định và tính toán các đại lượng cần thiết

> **Bước 4.a**: Kết luận dựa trên kết quả kiểm định

In [65]:
# Để kiểm định từng hàng, ta sử dụng vòng lặp for và so sánh giá trị p với mức ý nghĩa để tiến hành kiểm định
for i in range(0, 5, 1):
  p5 = st.shapiro(sample_converted_by_df5[i])[1]

  # Phát biểu kết quả kiểm định từng nhóm
  print("-------------------------------------------------------------------------")
  print(f"Giá trị p5 của nhóm thứ {i+1} là: {np.round(p5, 2)}")
  if (p5 < alpha):
    print(f"Vì giá trị p5 của nhóm thứ {i+1} bé hơn mức ý nghĩa của bài toán ({np.round(p5, 2)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
  else:
    print(f"Vì giá trị p5 của nhóm thứ {i+1} không bé hơn mức ý nghĩa của bài toán ({np.round(p5, 2)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

print("-------------------------------------------------------------------------")

-------------------------------------------------------------------------
Giá trị p5 của nhóm thứ 1 là: 0.03
Vì giá trị p5 của nhóm thứ 1 bé hơn mức ý nghĩa của bài toán (0.03 < 0.05). Vì vậy không có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p5 của nhóm thứ 2 là: 0.16
Vì giá trị p5 của nhóm thứ 2 không bé hơn mức ý nghĩa của bài toán (0.16 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p5 của nhóm thứ 3 là: 0.66
Vì giá trị p5 của nhóm thứ 3 không bé hơn mức ý nghĩa của bài toán (0.66 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p5 của nhóm thứ 4 là: 0.07
Vì giá trị p5 của nhóm thứ 4 không bé hơn mức ý nghĩa của bài toán (0.07 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-----------------------------------------------------------------

### b) Kiểm định sự đồng nhất giữa phương sai các nhóm

> **Bước 2.b**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Không có sự khác biệt về phương sai của các nhóm}$$

Giả thuyết $H_a$:

$$H_a: \text{Có sự khác biệt về phương sai của các nhóm}$$

> **Bước 3.b**: Tiến hành kiểm định và tính toán các đại lượng cần thiết


In [66]:
# Khởi tạo biến stt5, p5 để lưu giá trị của stat5 value và p5 value
stt5, p5 = st.levene(sample_converted_by_df5[0], sample_converted_by_df5[1], sample_converted_by_df5[2], sample_converted_by_df5[3], sample_converted_by_df5[4])   

# Xuất kết quả stat5 value, p5 value ra màn hình
print(f"Giá trị stat5: {np.round(stt5, 2)}")
print(f"Giá trị p5: {np.round(p5, 2)}")

Giá trị stat5: 0.1
Giá trị p5: 0.98


> **Bước 4.b**: Kết luận dựa trên kết quả kiểm định

In [67]:
if (p5 < alpha):
  print(f"Vì giá trị p5  bé hơn mức ý nghĩa của bài toán ({np.round(p5, 2)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
else:
  print(f"Vì giá trị p5 không bé hơn mức ý nghĩa của bài toán ({np.round(p5, 2)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

Vì giá trị p5 không bé hơn mức ý nghĩa của bài toán (0.98 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0


Vì các nhóm độc lập, tuân theo phân phối chuẩn và có sự đồng nhất về phương sai giữa các nhóm nên ta **có thể tiến hành kiểm định ANOVA**

### TIẾN HÀNH PHÂN TÍCH ANOVA 1 CHIỀU

> **Bước 1**: Lấy dữ liệu và tính toán các thông số cần thiết

*   Dữ liệu được sử dụng là sample_converted_by_df5 (Dữ liệu đã được xử lí từ owan05.csv)
*   alpha = 0.05 là ý nghĩa của bài toán trong quá trình kiểm định

> **Bước 2**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Không có sự khác biệt về trung bình độ tăng trưởng của các nhóm}$$

Giả thuyết $H_a$:

$$H_a: \text{có sự khác biệt về trung bình độ tăng trưởng của các nhóm}$$

> **Bước 3**: Tiến hành kiểm định ANOVA 1 chiều

In [68]:
# Khởi tạo 2 biến f5 và p5 dùng để lưu trữ giá trị f5 stat và p5 value
f5, p5 = st.f_oneway(sample_converted_by_df5[0], sample_converted_by_df5[1], sample_converted_by_df5[2], sample_converted_by_df5[3], sample_converted_by_df5[4])

# Xuất giá trị p5 ra màn hình
print(f"Giá trị p5 khi tiến hành kiểm định: {np.round(p5, 2)}")

Giá trị p5 khi tiến hành kiểm định: 0.19


> **Bước 4**: Kết luận dựa trên kết quả kiểm định

In [69]:
if (p5 < alpha):
  print(f"Vì giá trị p5  bé hơn mức ý nghĩa của bài toán ({np.round(p5, 2)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
else:
  print(f"Vì giá trị p5 không bé hơn mức ý nghĩa của bài toán ({np.round(p5, 2)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

Vì giá trị p5 không bé hơn mức ý nghĩa của bài toán (0.19 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
